# 06/13 Dummy

## Import & Load

In [146]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import localfunc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor


In [145]:
trn_df= pd.read_csv("./data/train_V2.csv")
tst_df = pd.read_csv("./data/test_V2.csv")

### Check (just in case)

In [27]:
test.isnull().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
dtype: int64

## Pre-Processing

In [257]:
#drop null col
for i in trn_df.columns.to_list() :
    trn_df.drop(index=trn_df[trn_df[i].isnull()==True].index, inplace=True)

In [258]:
#drop useless col
trn_df.drop("Id", axis=1, inplace=True)

In [259]:
#and save memory
trn_df = localfunc.reduce_ram_usage(trn_df)

### Feature Engineering

In [304]:
att_df = trn_df.copy()
att_df["item"] = att_df["boosts"] + att_df["heals"]
att_df

,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,...,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,item
0,4d4b580de459be,a10357fd1a4a91,0,0,0.00000,0,0,0,60,1241,...,0.000000,0,0.000000,0,0,244.7500,1,1466,0.444336,0
1,684d5656442f9e,aeb375fc57110c,0,0,91.50000,0,0,0,57,0,...,0.004501,0,11.039062,0,0,1434.0000,5,0,0.640137,0
2,6a4a42c3245a74,110163d8bb94ae,1,0,68.00000,0,0,0,47,0,...,0.000000,0,0.000000,0,0,161.7500,2,0,0.775391,0
3,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90625,0,0,0,75,0,...,0.000000,0,0.000000,0,0,202.7500,3,0,0.166748,0
4,de04010b3458dd,6dc8ff871e21e6,0,0,100.00000,0,0,0,45,0,...,0.000000,0,0.000000,0,0,49.7500,2,0,0.187500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446961,d238e426f50de7,18492834ce5635,0,0,0.00000,0,0,0,74,1029,...,1292.000000,0,0.000000,0,0,1019.0000,3,1507,0.178589,0
4446962,408cdb5c46b2ac,ee854b837376d9,0,1,44.15625,0,0,0,69,0,...,0.000000,0,0.000000,0,0,81.6875,6,0,0.293457,1
4446963,e26ac84bdf7cef,6d0cd12784f1ab,0,0,59.06250,0,0,0,66,0,...,0.000000,0,2.183594,0,0,788.5000,4,0,0.481445,0
4446964,c2223f35411394,c9c701d0ad758a,0,4,180.37500,1,1,2,11,0,...,0.000000,0,0.000000,0,0,2748.0000,8,0,0.799805,6


## Evaluation

In [305]:
#features = att_df[["walkDistance", "boosts", "heals", "kills"]]
features = att_df.drop(["groupId", "killPlace", "matchId","winPlacePerc", "matchType"], axis=1) #all

target = att_df["winPlacePerc"]
train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.2, random_state=141)

In [290]:
# scaler = MinMaxScaler()
# train_X = scaler.fit_transform(train_X)
# train_X = pd.DataFrame(train_X, columns=features.columns)
# test_X = scaler.fit_transform(test_X)
# test_X = pd.DataFrame(test_X, columns=features.columns)



In [306]:
model = LGBMRegressor()
model.fit(train_X, train_y)

LGBMRegressor()

In [307]:
pred_y = model.predict(test_X)

In [308]:
np.round(mean_absolute_error(pred_y, test_y) ,4)

0.0907

In [95]:
#base(4col) - default 0.1078
#base - outlier cut 0.1086

#base - default_line 0.1315
#base - out cut_line 0.1365



0.1086

오늘의 실험 결론:
>이상치 제거는   __정확도를 떨어트린다__


>컬럼 드랍은  __정확도를 떨어트린다__ (아무리 corr이 낮은 컬럼이더라도!)


>스케일링이  __정확도를 아주 약간 떨어트린다__ (다중공선성 제거에 필요하다면 활용할 수 있겠지만..?)


추가 결론:

>대부분의 형태의 추가는 +이다.


>대부분의 형태의 제거는 -이다.


>컬럼을 100종류씩 있는대로 만들어서 때려박고 하이퍼파라미터 튜닝을 한 후에 세부 조정을 해도 전혀 늦지 않을 것 같다.